In [258]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
def euclidean2(a, b):
    return sum((a - b) ** 2)

In [156]:
def gaussian_dist(dist2, sigma2=1):
    return np.exp(-dist2 / sigma2)

In [249]:
def knn(dataset, point, k=5, dist_func=euclidean2):
    dists = dataset.apply(dist_func, axis=1, b=point)
    k_nearest = dists.argsort()[:k]
    k_dists = dists[k_nearest].reset_index(drop=True)
    
    # Build dataframe
    ind_dist = pd.concat([k_nearest, k_dists], axis=1)
    ind_dist.columns = ["index", "dist2"]
    
    return ind_dist

In [250]:
def knn_classify(data, classes, point, **kwargs):
    knn_results = knn(data, point, **kwargs)
    knn_results["classes"] = classes[knn_results["index"]].values
    knn_results["gau"] = knn_results["dist2"].apply(gaussian_dist)
    
    knn_agg = knn_results.groupby("classes").sum()
    
    return knn_agg.idxmax()["gau"]

def knn_classify_many(data, classes, points, **kwargs):
    return points.apply(lambda p: knn_classify(data, classes, p, **kwargs), axis=1)

# Test data

In [150]:
_df = pd.DataFrame(columns=[0,1,2,3,4], 
                  data=np.random.random(size=(100,5)))

_point = np.random.random(size=(5,))
_points = pd.DataFrame(columns=[0,1,2,3,4], 
                      data=np.random.random(size=(5,5)))

_classes = pd.Series(np.random.randint(10, size=(100)))

In [164]:
knn(_df, _point)

,index,dist2
0,9,0.034699
1,91,0.175221
2,39,0.213508
3,7,0.238073
4,67,0.320056


In [163]:
knn_classify(_df, _classes, _point, k=11)

3

In [161]:
knn_classify_many(_df, _classes, _points, k=5)

0    4
1    6
2    5
3    7
4    4
dtype: int64

# Digits dataset

In [171]:
digits = datasets.load_digits()

In [173]:
print(digits["DESCR"])

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [212]:
train_data, test_data, train_classes, test_classes = train_test_split(digits["data"], digits["target"], test_size=0.2)
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
train_classes = pd.Series(train_classes)
test_classes = pd.Series(test_classes)

In [253]:
preds = knn_classify_many(train_data, train_classes, test_data)

In [252]:
preds

0    6
1    8
dtype: int64

In [265]:
confusion_matrix(test_classes, preds)

array([[36,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 29,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 32,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 38,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 43,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 37,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0, 39,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 32,  0,  0],
       [ 0,  1,  1,  0,  0,  1,  0,  0, 29,  0],
       [ 0,  0,  0,  2,  0,  0,  0,  0,  0, 39]], dtype=int64)